In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [2]:
from currency_converter import CurrencyConverter

## Open Geojson

In [3]:
czechia_gdf = gpd.read_file("data/geojson/Okresy_LAU_1_multi_20250701.geojson")

In [5]:
czech_to_english = {
    "Hlavní město Praha": "Prague, the Capital City",
    "Středočeský kraj": "Středočeský Region",
    "Jihočeský kraj": "Jihočeský Region",
    "Plzeňský kraj": "Plzeňský Region",
    "Karlovarský kraj": "Karlovarský Region",
    "Ústecký kraj": "Ústecký Region",
    "Liberecký kraj": "Liberecký Region",
    "Královéhradecký kraj": "Královéhradecký Region",
    "Pardubický kraj": "Pardubický Region",
    "Kraj Vysočina": "Vysočina Region",
    "Jihomoravský kraj": "Jihomoravský Region",
    "Olomoucký kraj": "Olomoucký Region",
    "Zlínský kraj": "Zlínský Region",
    "Moravskoslezský kraj": "Moravskoslezský Region",
}

In [6]:
czechia_gdf = czechia_gdf.dissolve(by="naz_kraj").reset_index()
czechia_gdf["region_english"] = czechia_gdf["naz_kraj"].map(czech_to_english)
czechia_gdf = czechia_gdf[["naz_kraj", "region_english", "geometry"]]

## Get Open Population data

In [7]:
wage_df = pd.read_excel("data/MZD07.xlsx", sheet_name=None, engine="openpyxl")

/mnt/f/Github/VineMapper/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [8]:
wage_df = (
    wage_df["DATA"][["Unnamed: 1", "Unnamed: 5"]].dropna().reset_index(drop=True)[1:]
)

In [9]:
wage_df = wage_df.rename(columns={"Unnamed: 1": "region_english", "Unnamed: 5": "WAGE"})

In [10]:
c = CurrencyConverter()
wage_df["WAGE"].astype(int)
wage_df["WAGE"] = wage_df["WAGE"].apply(
    lambda wage: int(round(c.convert(wage, "CZK", "EUR"), 0))
)

## Merge data and export

In [11]:
wage_gdf = czechia_gdf.merge(wage_df, on="region_english", how="left")

In [12]:
wage_gdf = wage_gdf.to_crs(5514)
wage_gdf.to_file("data/Avg_Monthly_Wage_Czechia_2024.gpkg")